JMJPFU
13-Mar-2020

Lord Bless this attempt of yours
#### Experimenting with autoencoders

In [1]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

In [4]:
!pip install opencv-python

In [6]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

In [7]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

In [8]:
# load the MNIST dataset
print("[INFO] loading MNIST dataset...")
((trainX, trainY), (testX, testY)) = mnist.load_data()

[INFO] loading MNIST dataset...


In [9]:
trainX.shape

(60000, 28, 28)

build our unsupervised dataset of images with a small amount of
contamination (i.e., anomalies) added into it


In [10]:
print("[INFO] creating unsupervised dataset...")

[INFO] creating unsupervised dataset...


In [20]:
def build_unsupervised_dataset(data, labels, validLabel=1,
	anomalyLabel=3, contam=0.01, seed=42):
	# grab all indexes of the supplied class label that are *truly*
	# that particular label, then grab the indexes of the image
	# labels that will serve as our "anomalies"
	validIdxs = np.where(labels == validLabel)[0]
	anomalyIdxs = np.where(labels == anomalyLabel)[0]
	# randomly shuffle both sets of indexes
	random.shuffle(validIdxs)
	random.shuffle(anomalyIdxs)
	# compute the total number of anomaly data points to select
	i = int(len(validIdxs) * contam)
	anomalyIdxs = anomalyIdxs[:i]
	# use NumPy array indexing to extract both the valid images and
	# "anomlay" images
	validImages = data[validIdxs]
	anomalyImages = data[anomalyIdxs]
	# stack the valid images and anomaly images together to form a
	# single data matrix and then shuffle the rows
	images = np.vstack([validImages, anomalyImages])
	np.random.seed(seed)
	np.random.shuffle(images)
	# return the set of images
	return images

In [21]:
images = build_unsupervised_dataset(trainX, trainY, validLabel=1,anomalyLabel=3, contam=0.01)

In [22]:
images.shape

(6809, 28, 28)

In [23]:
# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
images = np.expand_dims(images, axis=-1)
images = images.astype("float32") / 255.0
images.shape

(6809, 28, 28, 1)

In [24]:
# construct the training and testing split
(trainX, testX) = train_test_split(images, test_size=0.2,
	random_state=42)

In [26]:
testX.shape

(1362, 28, 28, 1)

#### construct our convolutional autoencoder

In [40]:
class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1
		# define the input to the encoder
		inputs = Input(shape=inputShape)
		x = inputs
		# loop over the number of filters
		for f in filters:
			# apply a CONV => RELU => BN operation
			x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# flatten the network and then construct our latent vector
		volumeSize = K.int_shape(x)
        #print('This is the volume size',volumeSize)
		x = Flatten()(x)
		latent = Dense(latentDim)(x)
		# build the encoder model
		encoder = Model(inputs, latent, name="encoder")
        # start building the decoder model which will accept the
		# output of the encoder as its inputs
		latentInputs = Input(shape=(latentDim,))
		x = Dense(np.prod(volumeSize[1:]))(latentInputs)
		x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
		# loop over our number of filters again, but this time in
		# reverse order
		for f in filters[::-1]:
			# apply a CONV_TRANSPOSE => RELU => BN operation
			x = Conv2DTranspose(f, (3, 3), strides=2,
				padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# apply a single CONV_TRANSPOSE layer used to recover the
		# original depth of the image
		x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
		outputs = Activation("sigmoid")(x)
		# build the decoder model
		decoder = Model(latentInputs, outputs, name="decoder")
		# our autoencoder is the encoder + decoder
		autoencoder = Model(inputs, decoder(encoder(inputs)),
			name="autoencoder")
		# return a 3-tuple of the encoder, decoder, and autoencoder
		return (encoder, decoder, autoencoder)

In [82]:
volumeSize

(None, 7, 7, 64)

In [41]:
print("[INFO] building autoencoder...")
(encoder, decoder, autoencoder) = ConvAutoencoder.build(28, 28, 1)


[INFO] building autoencoder...


In [42]:
encoder.summary()


Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 7, 7, 64)          256 

In [43]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3136)              53312     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_v2_4 (Ba (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 32)        1846

In [44]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 16)                69392     
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         109377    
Total params: 178,769
Trainable params: 178,385
Non-trainable params: 384
_________________________________________________________________


In [45]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mse", optimizer=opt)

In [46]:
# train the convolutional autoencoder
H = autoencoder.fit(
	trainX, trainX,
	validation_data=(testX, testX),
	epochs=EPOCHS,
	batch_size=BS)

Train on 5447 samples, validate on 1362 samples
Epoch 1/20
5447/5447 [==============================] - 21s 4ms/sample - loss: 0.0461 - val_loss: 0.0355
Epoch 2/20
5447/5447 [==============================] - 23s 4ms/sample - loss: 0.0132 - val_loss: 0.0064
Epoch 3/20
5447/5447 [==============================] - 24s 4ms/sample - loss: 0.0045 - val_loss: 0.0037
Epoch 4/20
5447/5447 [==============================] - 25s 5ms/sample - loss: 0.0034 - val_loss: 0.0034
Epoch 5/20
5447/5447 [==============================] - 26s 5ms/sample - loss: 0.0029 - val_loss: 0.0028
Epoch 6/20
5447/5447 [==============================] - 26s 5ms/sample - loss: 0.0027 - val_loss: 0.0028
Epoch 7/20
5447/5447 [==============================] - 27s 5ms/sample - loss: 0.0025 - val_loss: 0.0025
Epoch 8/20
5447/5447 [==============================] - 28s 5ms/sample - loss: 0.0023 - val_loss: 0.0023
Epoch 9/20
5447/5447 [==============================] - 31s 6ms/sample - loss: 0.0022 - val_loss: 0.0025
Epoch 1

In [48]:
print("[INFO] making predictions...")
decoded = autoencoder.predict(testX)

[INFO] making predictions...


In [49]:
def visualize_predictions(decoded, gt, samples=10):
	# initialize our list of output images
	outputs = None
	# loop over our number of output samples
	for i in range(0, samples):
		# grab the original image and reconstructed image
		original = (gt[i] * 255).astype("uint8")
		recon = (decoded[i] * 255).astype("uint8")
		# stack the original and reconstructed image side-by-side
		output = np.hstack([original, recon])
		# if the outputs array is empty, initialize it as the current
		# side-by-side image display
		if outputs is None:
			outputs = output
		# otherwise, vertically stack the outputs
		else:
			outputs = np.vstack([outputs, output])
	# return the output images
	return outputs

In [50]:
vis = visualize_predictions(decoded, testX)

In [51]:

cv2.imwrite('recon_vis.png', vis)

True

In [52]:
# serialize the image data to disk
print("[INFO] saving image data...")
f = open('Output/images.pickle', "wb")
f.write(pickle.dumps(images))
f.close()

[INFO] saving image data...


In [54]:
# serialize the autoencoder model to disk
print("[INFO] saving autoencoder...")
#tf.contrib.saved_model.save_keras_model(autoencoder,'Output/autoencoder.model')
autoencoder.save('Output/autoencoder.model.h5')

[INFO] saving autoencoder...


In [59]:
import tensorflow as tf
new_model = tf.keras.models.load_model('Output/autoencoder.model.h5')
new_model.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 16)                69392     
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         109377    
Total params: 178,769
Trainable params: 178,385
Non-trainable params: 384
_________________________________________________________________


### Creating the anomaly detector to detect anomalies

In [60]:
images = pickle.loads(open('Output/images.pickle', "rb").read())
images.shape

(6809, 28, 28, 1)

In [61]:
# make predictions on our image data and initialize our list of
# reconstruction errors
decoded = autoencoder.predict(images)
errors = []

In [62]:
errors = []
# loop over all original images and their corresponding
# reconstructions
for (image, recon) in zip(images, decoded):
	# compute the mean squared error between the ground-truth image
	# and the reconstructed image, then add it to our list of errors
	mse = np.mean((image - recon) ** 2)
	errors.append(mse)

In [79]:
# compute the q-th quantile of the errors which serves as our
# threshold to identify anomalies -- any data point that our model
# reconstructed with > threshold error will be marked as an outlier
thresh = np.quantile(errors, 0.98999)
idxs = np.where(np.array(errors) >= thresh)[0]
print("[INFO] mse threshold: {}".format(thresh))
print("[INFO] {} outliers found".format(len(idxs)))

[INFO] mse threshold: 0.009352713926210999
[INFO] 69 outliers found


In [80]:
idxs

array([  61,  104,  322,  409,  576,  734, 1056, 1150, 1166, 1197, 1215,
       1397, 1420, 1437, 1477, 1656, 1782, 1981, 1999, 2032, 2132, 2134,
       2263, 2268, 2302, 2567, 2819, 2897, 3010, 3044, 3074, 3168, 3204,
       3296, 3882, 3917, 3932, 4160, 4164, 4279, 4301, 4309, 4394, 4423,
       4523, 4543, 4575, 4600, 4806, 4925, 5129, 5226, 5283, 5306, 5446,
       5726, 5753, 5802, 5913, 5982, 6020, 6135, 6285, 6346, 6397, 6423,
       6440, 6624, 6736], dtype=int64)

In [81]:
# initialize the outputs array
outputs = None
# loop over the indexes of images with a high mean squared error term
for i in idxs:
	# grab the original image and reconstructed image
	original = (images[i] * 255).astype("uint8")
	recon = (decoded[i] * 255).astype("uint8")
	# stack the original and reconstructed image side-by-side
	output = np.hstack([original, recon])
	# if the outputs array is empty, initialize it as the current
	# side-by-side image display
	if outputs is None:
		outputs = output
	# otherwise, vertically stack the outputs
	else:
		outputs = np.vstack([outputs, output])
# show the output visualization
cv2.imshow("Output", outputs)
cv2.waitKey(0)


-1

### Experimenting with Layer visualization

In [47]:
autoencoder.inputs

[<tf.Tensor 'input_4:0' shape=(None, 28, 28, 1) dtype=float32>]